In [3]:
import numpy as np
import pandas as pd

# МAP

In [11]:
def grouper(df):
    return df.groupby('category').agg({
        'stars': ['mean', 'max','count'], 
        'price': ['mean', 'max']  
    }).reset_index()

def map(data):
    count = data['category'].nunique()
    dfs = np.array_split(data, count )
    grouped_data=[]
    for(df) in dfs:
        grouped_data.append(grouper(df))
    return grouped_data

In [30]:
data=pd.read_csv("./data/amz_us_price_prediction_dataset.csv") #Читаем данные
mapped=map(data)


C:\Users\Admin\PycharmProjects\HW1\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [33]:
mapp=pd.concat(mapped, ignore_index=True)
mapp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (category, )    598 non-null    object 
 1   (stars, mean)   598 non-null    float64
 2   (stars, max)    598 non-null    float64
 3   (stars, count)  598 non-null    int64  
 4   (price, mean)   598 non-null    float64
 5   (price, max)    598 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 28.2+ KB


# SHUFFLE

In [35]:
def shuffle(agg_df):
    if isinstance(agg_df.columns, pd.MultiIndex):
        agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
    categories = agg_df['category_'].unique()
    return [agg_df[agg_df['category_'] == cat].copy() for cat in categories]


data = pd.read_csv("./data/amz_us_price_prediction_dataset.csv")  #Читаем данные
mapped = map(data)
mapped()

In [36]:
shuffled=shuffle(mapp)

# REDUCE

In [61]:
def reduce(df):
    unique_categories = df['category_'].unique()
    if len(unique_categories) != 1:
        raise ValueError("В датафрейме более одной категории")
    
    weights = df['stars_count']
    total_weight = weights.sum()
    
    weighted_stars_mean = (df['stars_mean'] * weights).sum() / total_weight
    weighted_price_mean = (df['price_mean'] * weights).sum() / total_weight
    

    max_stars = df['stars_max'].max()
    max_price = df['price_max'].max()

    result_df = pd.DataFrame({
        'Категория': [unique_categories[0]],
        'Сред рейтинг': [weighted_stars_mean],
        'Средняя цена': [weighted_price_mean],
        'Макс рейтинг': [max_stars],
        'Макс цена': [max_price]
    })
    
    return result_df

In [62]:
FinalList=[]
for df in shuffled:
    CategoryFinal=reduce(df)
    FinalList.append(CategoryFinal)


In [64]:
finalDataframe=pd.concat(FinalList, ignore_index=True)
finalDataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Категория     236 non-null    object 
 1   Сред рейтинг  236 non-null    float64
 2   Средняя цена  236 non-null    float64
 3   Макс рейтинг  236 non-null    float64
 4   Макс цена     236 non-null    float64
dtypes: float64(4), object(1)
memory usage: 9.3+ KB


In [65]:
with pd.option_context('display.max_rows', None):  # None means unlimited
    display(finalDataframe)

,Категория,Сред рейтинг,Средняя цена,Макс рейтинг,Макс цена
0,Men's Clothing,4.241175,45.005594,5.0,769.00
1,Suitcases,3.491558,122.138139,5.0,1495.00
2,Consoles & Accessories,3.803882,34.478278,5.0,10010.00
3,Men's Shoes,3.972481,93.576368,5.0,889.06
4,Men's Accessories,4.237318,42.761997,5.0,605.00
5,Televisions & Video Products,4.312991,41.606123,5.0,4565.61
6,Vacuum Cleaners & Floor Care,4.251730,46.503754,5.0,1760.00
7,Additive Manufacturing Products,3.106792,53.640182,5.0,9009.00
8,Headphones & Earbuds,3.709447,52.659478,5.0,4999.00
9,Boys' Watches,3.500000,18.324675,5.0,65.82
